In [15]:
import os
os.chdir("/root/rag_project")
os.getcwd()

'/root/rag_project'

In [16]:
import json
import pandas as pd
from utils import file_control
from utils import inference
from train import colbert_train
import infer

## Reranking Inference

In [17]:
eval_list =  file_control.read_jsonl("data/eval_query_gpt4.jsonl")
docs = file_control.read_jsonl("data/documents.jsonl")
index = "multilingual-colbert-filter-nbit8"
main_path = "sc_documents"

# Dongjin-kr/ko-reranker
model_name = "models/ko_reranker"
output_path = "outputs/rerank_result_test01.jsonl"

In [18]:
search_list = colbert_train.make_search_result_faiss(eval_list, docs, index, 20, main_path)

[May 07, 18:59:29] #> Loading codec...
[May 07, 18:59:29] #> Loading IVF...
[May 07, 18:59:29] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 4219.62it/s]

[May 07, 18:59:29] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 33.04it/s]


In [19]:
infer.evaluate_reranker(search_list, model_name, output_path, 64)

Load Model and Tokenizer


Reranking: 100%|██████████| 64/64 [01:08<00:00,  1.07s/it]


Save rerank result at outputs/rerank_result_test01.jsonl
Rerank vaild mAP Score:  0.8523102310231022


## 결과 분석

### None-Finetune Reranker

In [3]:
none_tune_res = file_control.read_jsonl("outputs/rerank_none_finetune01.jsonl")
docs_df = file_control.read_jsonl("data/documents.jsonl", True)

In [4]:
# 라벨링한 정답 가져오기
def keystoint(x):
    return {int(k): v for k, v in x.items()}

with open("data/gt_eval_dict.json", 'r') as f:
    gt_eval_dict = json.load(f, object_hook=keystoint)

pred_df = pd.DataFrame(none_tune_res).loc[0:100]
gt_df = file_control.read_jsonl("data/gt_eval.jsonl", to_csv=True)
none_tune_df = pred_df.merge(gt_df, how='left', on='eval_id')
none_tune_df['score'] = inference.calc_map(gt_eval_dict, none_tune_res[0:101], get_each_result=True)

mAP Score:  0.8770627062706269


In [5]:
inference.print_details2(none_tune_df.loc[0], docs_df)

## eval_id: 78
user msg: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
standalone_query: 나무 분류 방법 조사
score: 1.0
정답 문서의 갯수: 3
==================== PRED ====================
*✅ 1 c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d: 0.0214004256
한 학생이 다양한 종류의 나무를 조사하고 있습니다.
 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다.
 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다.
 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다.
 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다.
 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다.
 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.

* 2 4f11bc9b-1b9c-47f1-8600-bcdf78db5b92: 6.5094e-06
메인주의 발삼전나무는 바늘잎 길이가 비슷합니다.
 이는 씨앗 내부의 유전 정보 때문입니다.
 발삼전나무의 씨앗은 유전 정보를 가지고 있으며, 이 정보는 바늘잎의 길이에 영향을 미칩니다.
 따라서, 발삼전나무의 씨앗이 비슷한 유전 정보를 가지고 있기 때문에 바늘잎의 길이도 비슷하게 나타납니다.
 이러한 특성은 발삼전나무를 식별하는데 도움을 줄 수 있습니다.

* 3 9712bdf6-9419-4953-a8f1-8a4015dee986: 4.9955e-06
생물학에서 일부 생물체의 분류 방법이 변경되었습니다.
 이제 생물체를 재분류하는 데에는 구조보다는 분자 수준에서의 조사가 사용됩니다.
 이 새로운 방법은 생물체의 유전자나 단백질의 구조와 기능을 분석하여 그들의 진화적 관계를 밝히는 데에 큰 도움이 됩니

### Finetune Reranker

In [6]:
tune_res = file_control.read_jsonl("outputs/rerank_finetune01.jsonl")
docs_df = file_control.read_jsonl("data/documents.jsonl", True)

In [7]:
# 라벨링한 정답 가져오기
def keystoint(x):
    return {int(k): v for k, v in x.items()}

with open("data/gt_eval_dict.json", 'r') as f:
    gt_eval_dict = json.load(f, object_hook=keystoint)

pred_df = pd.DataFrame(tune_res).loc[0:100]
gt_df = file_control.read_jsonl("data/gt_eval.jsonl", to_csv=True)
tune_df = pred_df.merge(gt_df, how='left', on='eval_id')
tune_df['score'] = inference.calc_map(gt_eval_dict, tune_res[0:101], get_each_result=True)

mAP Score:  0.8457095709570956


In [8]:
inference.print_details2(tune_df.loc[0], docs_df)

## eval_id: 78
user msg: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
standalone_query: 나무 분류 방법 조사
score: 1.0
정답 문서의 갯수: 3
==================== PRED ====================
*✅ 1 c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d: 0.1734207422
한 학생이 다양한 종류의 나무를 조사하고 있습니다.
 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다.
 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다.
 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다.
 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다.
 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다.
 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.

* 2 9712bdf6-9419-4953-a8f1-8a4015dee986: 2.4e-09
생물학에서 일부 생물체의 분류 방법이 변경되었습니다.
 이제 생물체를 재분류하는 데에는 구조보다는 분자 수준에서의 조사가 사용됩니다.
 이 새로운 방법은 생물체의 유전자나 단백질의 구조와 기능을 분석하여 그들의 진화적 관계를 밝히는 데에 큰 도움이 됩니다.
 이러한 분자 수준의 조사는 생물체의 유전적 유사성을 파악하고, 서로 다른 종 간의 진화적 연결고리를 찾는 데에 중요한 역할을 합니다.
 이 방법은 생물체의 분류를 더욱 정확하고 명확하게 만들어주며, 생물 다양성 연구에도 큰 기여를 하고 있습니다.
 이제 구조보다는 분자 수준에서의 조사가 생물체의 재분류에 사용되고 있으며, 이는 생물학의 발전에 새로운 지평을 열어주고 있습니다.

* 3 a3071404-122f-408b-86a0-1787b4449a1b